In [4]:
import pandas as pd
# mlxtend: machine learning extensions
# TransactionEncoder alışveriş sepeti gibi liste şeklindeki verileri [['milk', 'bread'], ['bread']] binary tabloya çevirir.
from mlxtend.preprocessing import TransactionEncoder 
# apriori: Sık geçen ürün gruplarını (frequent itemsets) bulmak için kullanılan bir fonksiyondur.
# association_rules: Bu sık gruplardan anlamlı ilişki kuralları (X->Y) üreten bir fonksiyondur.
# Örneğin, {milk, bread} sık geçiyor milk->bread kuralı çıkarılabilir.
from mlxtend.frequent_patterns import apriori, association_rules
# Logging: Uygulama içinde hata ve bilgi mesajlarını loglamak (kaydetmek) için kullanılır.
import logging

|Yapı |Örnek |Terim|
|:-----|----|----:|
|Modül | `mlxtend.frequent_patterns` |Python modülü (module)|
|Fonksiyon | `apriori()`,`association_rules()` |Python fonksiyonu (function)|
|Paket | `mlxtend` |Python paketi (package)|
|Kütüphane | `mlxtend` |Python kütüphanesi (library)|

- **Paket:** Birden fazla modülü içeren Python klasörüdür.
- **Modül:** Bir veya birkaç fonksiyon, sınıf ve değişken içeren `.py` uzantılı Python dosyasıdır.
- **Kütüphane:** Belirli bir amacı gerçekleştirmek için hazırlanmış, modül ve paketlerden oluşan bütün yapıdır.
- **Fonksiyon:** Belirli bir işlemi gerçekleştiren, çağrılabilir kod bloğudur.

In [5]:
# 1. Veri Yükleme ve Ön İşleme
def load_transactions(csv_path):
    try:
        df = pd.read_csv(csv_path, header=None) #header=None: Veri dosyasında sütun başlığı olmadığı için 
        # Satırları listeye çevirme, her satır bir alışverişi (transaction) temsil eder.
        transactions = df.apply( # Pandas DataFrame'inin (df) her satırı için bir işlem yapar.
            lambda row: [str(item).strip().lower() for item in row if pd.notnull(item)],
            axis=1 # satır bazlı işlem demektir (yani her alışveriş sepeti bir satırdır).
        ).tolist() 
        return transactions
    except Exception as e:
        logging.error(f"Veri okunamadı: {e}") # Hata `logging` modülüyle kaydedilir (terminalde veya log dosyasında görünür).
        return [] # Fonksiyon boş liste döndürür ki sonraki adımlar çökmesin.

## transactions
- `df.apply(..., axis=1)`: pandas DataFrame'in her satırını tek tek işler. Her satır bir pandas `Series` objesi olarak `lambda` fonksiyonuna gönderilir.
- `row` her bir satırı temsil eder.
- `row` içindeki tüm ürünler:
    - `str(item)`: Sayı bile olsa stringe çevrilir
    - `.strip()`: Kenarlardaki boşluklar temizlenir.
    - `.lower()`: Küçük harfe çevrilir ("Milk" ≠ "milk" problemi önlenir).
    - `if pd.notnull(item)`: Boş hücreler atılır (CSV boşlukları `NaN` olur).
- `lambda row: [str(item).strip().lower() for item in row if pd.notnull(item)]` list comprehension (liste oluşturma) yapısıdır. Okuma sırası:
1. `for item in row`
2. `if pd.notnull(item)` # Eğer item boş değilse (NaN) değilse
3. `str(item).strip().lower()` # Temizlenmiş string olarak listeye ekle
- Yani önce döngü, sonra koşul, sonra dönüştürme işlemi çalışır.
- Örnek:
    - " Milk ", "BREAD", None
    - -> ["milk", "bread"]

In [6]:
# 2. One-Hot Encoding
def preprocess_transactions(transactions):
    te = TransactionEncoder() # TransactionEncoder() sınıfından bir nesne oluştur (mlxtend kütüphanesinden)
    te_array = te.fit(transactions).transform(transactions) # Ürün isimlerini öğren ve sepet verilerini 0/1 (True/False) matrise çevir
    df_transformed = pd.DataFrame(te_array, columns=te.columns_) # Elde edilen matrisi pandas DataFrame'e dönüştür, sütun adları ürün isimleri
    return df_transformed # Sonuç olarak one-hot encoded veri çerçevesini (0/1 tablosu) döndür

## One-Hot Encoding
- transactions = [['milk','bread'],['bread']]
- preprocess_transactions(transactions) fonksiyonu çalıştıktan sonra
- df_transformed =
| |bread |milk|
|:-----|----|----:|
|0 |True |True|
|1 |True |False|
- Bu çıktı, her müşterinin hangi ürünü alıp almadığını binary formatta gösterir. Bu format, Apriori Algoritması için gereklidir.

In [7]:
# 3. Apriori Uygulama
# Apriori Algoritmasını Çalıştır:
# df_transformed: one-hot encoded veri (0/1) tablosu
# min_support: en az yüzde kaç destek alan ürün grupları alınsın?
# use_colnames=True: sonuçlarda ürün indeksleri yerine isimlerini göster
def run_apriori(df_transformed, min_support=0.02):
    frequent_itemsets = apriori(df_transformed, min_support=min_support, use_colnames=True)
    return frequent_itemsets # Sık geçen ürün kümelerini (itemset'leri) içeren tabloyu döndür.

In [8]:
# 4. Kuralların Çıkarılması
def extract_rules(frequent_itemsets, min_confidence=0.3, min_lift=1.2):
    # Apriori algoritması sonucu bulunan sık itemset'lerden ilişki kuralları üret
    # metric="lift": kural gücünü değerlendirmek için lift kullanılır
    # min_threshold=min_lift: lift için eşik belirlenir.
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)
    # Üretilen kurallar arasından güven (confidence) ve lift değeri yeterli olanları filtrele
    filtered_rules = rules[
        (rules['confidence'] >= min_confidence) &
        (rules['lift'] >= min_lift)
    ]

    # Geriye sadece anlamlı olan kuralların temel metrikleri döndürülür:
    # antecedents: Eğer kısmı (X → Y kuralındaki X)
    # consequents: Sonuç kısmı (X → Y kuralındaki Y)
    # support, confidence, lift: değerlendirme metrikleri
    return filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [9]:
# Ana Çalıştırma Fonksiyonu
def main(csv_path):
    # 1. Veriyi CSV'den oku ve her satırı bir alışveriş sepetine çevir (liste listesi)
    transactions = load_transactions(csv_path)
    # 2. Sepet verisini 0/1 formatına (one-hot encoding) dönüştür
    df_trans = preprocess_transactions(transactions)
    # 3. Apriori algoritmasını kullanarak sık geçen ürün kümelerini (itemset) bul
    itemsets = run_apriori(df_trans, min_support=0.02)
    # 4. Bu sık kümelerden ilişki kurallarını çıkar ve filtrele (confidence ve lift'e göre)
    rules = extract_rules(itemsets, min_confidence=0.3, min_lift=1.2)
    # 5. Anlamlı kuralları döndür
    return rules

In [10]:
# Kod bu dosya çalıştırıldığında aktif olur.
if __name__ == "__main__": #Kod hem modül olarak başka yerden çağrılabilir, hem de doğrudan terminalden çalıştırılabilir olur.
    csv_dosya = "sepet.csv"
    anlamli_kurallar = main(csv_dosya)
    print(anlamli_kurallar)
    anlamli_kurallar.to_csv("anlamli_kurallar.csv",index=False)

# if __name__ == "__main__": bloğu eğer başka bir Python dosyasından import edilirse çalışmaz.
# Başka dosyada import edilirse; sadece fonksiyonlar çalışır, otomatik işlem yapılmaz.

            antecedents      consequents   support  confidence      lift
0             (burgers)           (eggs)  0.028796    0.330275  1.837830
6                (cake)  (mineral water)  0.027463    0.338816  1.421397
8             (chicken)  (mineral water)  0.022797    0.380000  1.594172
18          (chocolate)  (mineral water)  0.052660    0.321400  1.348332
22        (cooking oil)  (mineral water)  0.020131    0.394256  1.653978
34    (frozen smoothie)  (mineral water)  0.020264    0.320000  1.342461
38  (frozen vegetables)  (mineral water)  0.035729    0.374825  1.572463
44        (ground beef)  (mineral water)  0.040928    0.416554  1.747522
46        (ground beef)      (spaghetti)  0.039195    0.398915  2.291162
48     (low fat yogurt)  (mineral water)  0.023997    0.313589  1.315565
51               (milk)  (mineral water)  0.047994    0.370370  1.553774
54          (olive oil)  (mineral water)  0.027596    0.419028  1.757904
57           (pancakes)  (mineral water)  0.033729 

In [11]:
# TODO: OOP prensiplerine uygun hale getirilecek.